In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

Systematic Sampling Example

In [2]:
np.random.seed(42)

data = pd.DataFrame({
    'ID':range(1,101),
    'Değer':np.random.normal(100,15,100)
})

def sistematik_ornekleme(df,orneklem_buyuklugu):
    N = df.shape[0]
    k = N // orneklem_buyuklugu

    baslangic = np.random.randint(1, k+1)
    secilen_indeksler = range(baslangic-1, N, k)
    return df.iloc[secilen_indeksler]

orneklem_buyuklugu=20
orneklem = sistematik_ornekleme(data, orneklem_buyuklugu)

print("Orijinal veri setinin büyüklüğü:", data.shape[0])
print("Örneklem büyüklüğü:",orneklem.shape[0])
print("\nİlk 5 örneklem:\n", orneklem.head())

Orijinal veri setinin büyüklüğü: 100
Örneklem büyüklüğü: 20

İlk 5 örneklem:
     ID       Değer
0    1  107.450712
5    6   96.487946
10  11   93.048735
15  16   91.565687
20  21  121.984732


Stratified and Weighted random sampling

Tabakalı(Startified) Örnekleme: Tabakalı örnekleme, popülasyonu belirli özelliklere göre gruplara ayırıp, her tabakadan ayrı ayrı örneklem alma yöntemidir. Bu yöntem, popülasyondaki alt grupların temsilini garanti eder. Örneğin:
- Cinsiyet bazında örnekleme(Kadın/Erkek)
- Yaş gruplarına göre örnekleme
- Eğitim seviyesine göre örnekleme

1. Tabakalı Örnekleme(Stratified Sampling)
   - Önce her tabakadi oranlar hesaplanır
   - Bu oranlara göre her tabakadan kaç örnek alınacağını belirler
   - Her tabakadan belirlenen sayıda rastgele örnek alır
   - Örnekleri birleştirip final örneklemi oluşturur
2. Ağırlıklı Örnekleme(Weighted Sampling)
   - Ağırlıkları normalize eder(toplam 1 olacak şekilde)
   - pandasın sample fonksiyonunu weights parametresi ile kullanır
   - Belirtilen büyüklükte ağırlıklı örneklem oluşturur
   

In [3]:
np.random.seed(42)

N=1000 #Popülasyon büyüklüğü
data = pd.DataFrame({
    'yaş':np.random.normal(35,12,N).astype(int),
    'cinsiyet':np.random.choice(['Erkek','Kadın'],N),
    'gelir':np.random.normal(5000,2000,N),
    'eğitim':np.random.choice(['ilköğretim','ortaöğretim','yükseköğretim'])
})

data['yaş_grubu'] = pd.cut(
    data['yaş'],
    bins=[0,25,35,45,100],
    labels=['18-25','26-35','36-45','45+']
)

In [4]:
def tabakali_ornekleme(df,tabaka_sutunu,ornekleme_buyuklugu):
    tabaka_oranlari = df[tabaka_sutunu].value_counts(normalize=True)
    tabaka_orneklem_sayilari = (tabaka_oranlari*ornekleme_buyuklugu).round().astype(int)
    tabakali_orneklem = pd.DataFrame()
    for tabaka, n in tabaka_orneklem_sayilari.items():
        tabaka_verisi = df[df[tabaka_sutunu]==tabaka]
        tabaka_orneklemi = tabaka_verisi.sample(n=n, random_state=42)
        tabakali_orneklem = pd.concat([tabakali_orneklem, tabaka_orneklemi])
    return tabakali_orneklem
tabakali_orneklem = tabakali_ornekleme(data,'yaş_grubu',200)
print("\nOrijinal veri seti yaş grubu dağılımı:", data['yaş_grubu'].value_counts(normalize=True))
print("\nTabakalı örneklem yaş grubu dağılımı:", tabakali_orneklem['yaş_grubu'].value_counts(normalize=True))


Orijinal veri seti yaş grubu dağılımı: yaş_grubu
26-35    0.309309
36-45    0.299299
18-25    0.218218
45+      0.173173
Name: proportion, dtype: float64

Tabakalı örneklem yaş grubu dağılımı: yaş_grubu
26-35    0.308458
36-45    0.298507
18-25    0.218905
45+      0.174129
Name: proportion, dtype: float64


In [5]:
def agirlikli_ornekleme(df,agirlik_sutunu,orneklem_buyuklugu):
    df[agirlik_sutunu] = df[agirlik_sutunu].apply(lambda x: np.nan if x <= 0 else x)
    df[agirlik_sutunu] = df[agirlik_sutunu].fillna(df[agirlik_sutunu].mean())
    agirliklar = df[agirlik_sutunu] / df[agirlik_sutunu].sum()
    orneklem = df.sample(n=orneklem_buyuklugu, weights = agirliklar, random_state=42)
    return orneklem

agirlikli_orneklem = agirlikli_ornekleme(data, 'gelir', 200)
print("\nOrijinal veri seti gelir ortalaması:", data['gelir'].mean())
print("Agırlıklı örneklem gelir ortalaması:", agirlikli_orneklem['gelir'].mean())


Orijinal veri seti gelir ortalaması: 5104.611031877225
Agırlıklı örneklem gelir ortalaması: 5858.829930786858


In [ ]:
#Pratik Alıştırmalar
#Öncelikle tabakalı örnekleme için kodlar yeniden yazılmalısın
def tabakali_ornekleme(df, tabaka_sutunu, orneklem_buyuklugu):
    
#1. Farklı bir sütuna göre tabakalı örnekleme yapın